<a href="https://colab.research.google.com/github/HackElite-FYP/Legal-Research-Platform-Core/blob/main/colab-main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GitHub Commands**

In [15]:
# @title GitHub Init
from google.colab import userdata

GH_UNAME = userdata.get('GH_UNAME')
GH_APIKEY = userdata.get('GH_APIKEY')
GH_EMAIL = userdata.get('GH_EMAIL')
PRIMARY_REPO_NAME = 'Legal-Research-Platform'
LOCAL_REPO_DIR = '/content/drive/MyDrive/FYP/GitHub/Legal-Research-Platform'

!git config --global user.name {GH_UNAME}
!git config --global user.email {GH_EMAIL}

%cd {LOCAL_REPO_DIR}

/content/drive/MyDrive/FYP/GitHub/Legal-Research-Platform


In [2]:
# @title Git <-
!git fetch

!git pull

Already up to date.


In [16]:
# @title Git ->
# !git add .

# !git status

# !git commit -m 'updated layout'

!git push

fatal: could not read Username for 'https://github.com': No such device or address


# **Scripts**